In [7]:
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import pandas as pd
import datetime
import time
import os

In [8]:
def get_week_dates(year, week_number):
    # Calculate the first day of the year
    first_day_of_year = datetime.datetime(year, 1, 1)
    # Calculate the start date of the week
    start_date = first_day_of_year + datetime.timedelta(weeks=week_number - 1)
    # Adjust the start date to the first day of the week (Monday)
    start_date = start_date - datetime.timedelta(days=start_date.weekday())
    # Calculate the end date of the week (Sunday)
    end_date = start_date + datetime.timedelta(days=6)
    return start_date.date(), end_date.date()

In [16]:
def get_month_dates(year, month_number):
    # Calculate the first day of the month
    first_day_of_month = datetime.datetime(year, month_number, 1)
    # Calculate the last day of the month
    if month_number == 12:
        last_day_of_month = datetime.datetime(year + 1, 1, 1) - datetime.timedelta(days=1)
    else:
        last_day_of_month = datetime.datetime(year, month_number + 1, 1) - datetime.timedelta(days=1)
    return first_day_of_month.date(), last_day_of_month.date()

In [9]:
def query_trino_admin(query_string: str, is_prod: bool = True):
    cfg = {
        'host': 'trino.telegraphbay.app' if is_prod else 'trino.staging.telegraphbay.app',
        'port': '443',
        'user': 'flora',
        'password': 'vmt-qyh*fxu*ufb2HFW',
        'catalog': 'iceberg',
        'schema': 'animoca',
    }
    conn = connect(
        host=cfg['host'],
        port=cfg['port'],
        user=cfg['user'],
        auth=BasicAuthentication(cfg['user'], cfg['password']),
        http_scheme="https",
        catalog=cfg['catalog'],
        schema=cfg['schema']
    )
    cur = conn.cursor()
    cur.execute(query_string)
    columns=[]
    for i in range(len(cur.description)):
        columns.append(cur.description[i][0])
    try:
        rows = cur.fetchall()
        result = pd.DataFrame(rows, columns=columns)
        cur.close()
        conn.close()
        print('Query successful')
        return result
    except Exception as e:
        print(e)
        return None

In [19]:
if __name__ == '__main__':
    chain = 'bsc'
    year = 2024
    # for i in range(2, 42):
    #     week_number = i
    #     start_date, end_date = get_week_dates(year, week_number)
    #     print(f"Week {week_number} of {year} starts on {start_date} and ends on {end_date}.")
    for i in range(1, 10):
        month_number = i
        start_date, end_date = get_month_dates(year, month_number)
        print(f"Month {month_number} of {year} starts on {start_date} and ends on {end_date}.")
        # query string
        query_string = f"""WITH 
            token_list as ( 
                select 
                    token_address 
                    from token_info 
                where token_symbol = 'APE'
            )
            ,  wallets_exclude as (
                select
                *
                from iceberg.prod_silver.treasury_wallet_labels
                where project in (
                    select
                    project
                    from project_mapping
                    where protocol_slug = (
                        select
                        max(protocol_slug)
                        from
                        contract_info
                        where contract_address in  (select token_address from token_list )
                        )
                    )
                )
            , current_holding as (
                select
                    wallet_address
                    , amount
                    from (
                        select
                        chain,
                        wallet_address,
                        token_address,
                        amount
                        from iceberg.animoca.address_token_latest_balance
                        where 1 = 1
                        and token_address in (select token_address from token_list)
                        ) a
                    where amount > 0
            )
            select 
                date_trunc('day', block_timestamp) as date,
                to_address,
                from_address,
                count(1) as tx_count
            from {chain}_transactions_full
            where 1 = 1
                and input = '0x'
                and ((from_address in (select wallet_address from current_holding)) or (to_address in (select wallet_address from current_holding)))
                and from_address not in (select address from address_attributes)
                and to_address not in (select address from address_attributes)
                and block_timestamp >= timestamp '{start_date}'
                and block_timestamp < timestamp '{end_date}'
                and block_timestamp < current_timestamp
                and block_timestamp >= timestamp '2024-01-01'
            group by 1, 2, 3
            """

        result = query_trino_admin(query_string)
        # save result to csv
        print('Saving result to csv...')
        # create a folder called result if it does not exist
        if not os.path.exists('ape_holder_data_fetch_result'):
            os.makedirs('ape_holder_data_fetch_result')
        result.to_csv(f'ape_holder_data_fetch_result/result_m{month_number}_{chain}.csv', index=False)
        print(f'Done saving result to csv for {chain}, month {month_number}')
        # wait for 10 seconds
        time.sleep(10)
    print('All done')

Month 1 of 2024 starts on 2024-01-01 and ends on 2024-01-31.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 1
Month 2 of 2024 starts on 2024-02-01 and ends on 2024-02-29.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 2
Month 3 of 2024 starts on 2024-03-01 and ends on 2024-03-31.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 3
Month 4 of 2024 starts on 2024-04-01 and ends on 2024-04-30.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 4
Month 5 of 2024 starts on 2024-05-01 and ends on 2024-05-31.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 5
Month 6 of 2024 starts on 2024-06-01 and ends on 2024-06-30.
Query successful
Saving result to csv...
Done saving result to csv for bsc, month 6
Month 7 of 2024 starts on 2024-07-01 and ends on 2024-07-31.
Query successful
Saving result to csv...
Done saving result to csv fo